In [1]:
import pandas as pd
from datetime import datetime
import requests
import time 
from requests_html import HTMLSession
from PIL import Image
from io import StringIO, BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output


In [3]:
! pip install --upgrade selenium

In [2]:
session = HTMLSession()

In [2]:
start = time.time()
pages = 100
single_page_urls = []
page_cnt, url_cnt = 0, 0
for p in range(1, pages+1):
    bikroy_url = f'https://bikroy.com/en/ads/bangladesh/home-living?sort=date&order=desc&buy_now=0&urgent=0&page={p}'
    print(bikroy_url)
    r = session.get('http://localhost:8050/render.html', params={'url': bikroy_url, 'wait': 1})

    page_cnt += 1
    for i in range (1, 28):
        try:
#             single_url = driver.find_element(by=By.XPATH, value=f'//*[@id="app-wrapper"]/div[1]/div[3]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/ul/li[{i}]/a').get_attribute('href')
            single_url = r.html.find(f'#app-wrapper > div.container--297Nx.all > div:nth-child(3) > div > div.row--3Vhjr.main-container--28aG5.justify-content-flex-start--1Xozy.align-items-normal--vaTgD.flex-wrap-wrap--2PCx8.flex-direction-row--27fh1.flex--3fKk1 > div.sm-col-12--30zDS.lg-col-9--20qCf.block--3v-Ow > div.ad-list-container--1UnyA > div > div.ad-list--2Y3ql > div.list-wrapper--t_A02 > ul > li:nth-child({i}) > a')
            single_url = single_url[0].attrs['href']
            single_page_urls.append(single_url)
            url_cnt += 1
            print("Page: " + str(page_cnt) + " URL: " + str(url_cnt))
        except:
            print('Not details link')
    
    clear_output(wait=True)
print(time.time()-start)
# driver.maximize_window()

524.0431141853333


In [5]:
len(single_page_urls), single_page_urls

(2673,
 ['/en/ad/suzuki-solio-g-edition-push-start-2011-for-sale-dhaka',
  '/en/ad/toyota-noah-x-sun-roof-octane-2004-for-sale-dhaka',
  '/en/ad/baaccaader-dui-tti-saaikel-bikriyy-kraa-hbe-for-sale-dhaka',
  '/en/ad/tvs-metro-kono-kaaj-naai-2013-for-sale-khulna-division',
  '/en/ad/ttaakaa-pryyojn-taai-sel-dib-for-sale-mymensingh',
  '/en/ad/dayang-dy-150-dayun-2013-for-sale-khulna',
  '/en/ad/tvs-apache-rtr-4v-dd-sohaa-mttrs-2020-for-sale-dhaka-division-4',
  '/en/ad/mitsubishi-outlander-octane-draive-2008-for-sale-dhaka-1',
  '/en/ad/hero-hunk-double-disk-2018-for-sale-chattogram-division',
  '/en/ad/rcb-s1-master-brake-pump-for-sale-dhaka',
  '/en/ad/tvs-apache-rtr-2019-for-sale-dhaka-division-304',
  '/en/ad/beetle-bolt-alligator-good-2021-for-sale-dhaka-division',
  '/en/ad/wuyung-ad-80s-2014-for-sale-khulna-division',
  '/en/ad/toyota-dyna-pick-up-dhaka-ready-2016-for-sale-dhaka',
  '/en/ad/toyota-corolla-111-1998-for-sale-chattogram-65',
  '/en/ad/walton-fusion-2017-2015-for-sal

In [4]:
with open('category_url/home-living_product_urls.txt', 'w') as f:
    for s in single_page_urls:
        f.write(s)
        f.write('\n')

In [36]:
def title_scrap(r):
#     try:
        print(r.status_code)
        title = r.html.find('.title--3s1R8')[0].text
        print(title) #[0].text
        print("Title: ", title)
        return title        
#     except:
        print("Product Not found. May be sold out!")
        return None

In [37]:
def time_location_scrap(r):
    time_location = r.html.find('.sub-title--37mkY')[0].text
    print("Time & Location: ", time_location)

    return time_location

In [38]:
def price_scrap(r):
    price = r.html.find('.amount--3NTpl')[0].text
    print("Price: ", price)

    return price

In [57]:
def image_scrap(r, category):
#     try:
        images = r.html.xpath('//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[1]/div/div/ul')
        print(images)
        img_path = []
        for img in images.find_elements(by=By.TAG_NAME,value='img'):
            img_urls = img.get_attribute('src')

            img_urls_firstname = '/'.join(img_urls.split('/')[:-3])
            img_urls_rename = img_urls_firstname + '/620/466/fitted.jpg'
    #         print(img_urls_rename)
            img_name = '@'.join(img_urls_firstname.split('/')[-2:])

            r = requests.get(img_urls_rename)
            i = Image.open(BytesIO(r.content))
            i.save(f"img/{category}/{img_name}.jpg")
            img_path.append(img_name)
        return img_path
#     except:
#         print('No images')        
#         return 'N/A'

In [71]:
# r, category
s = 'https://bikroy.com/en/ad/oppo-a57-3-32-new-for-sale-barishal-7'
r = session.get('http://localhost:8050/render.html', params={'url': s, 'wait': 3})
txt = r.html.text.split('initialData = ')[1]
images = txt.split('"image:"')
print(images)
# print(r.html.text.split('initialData = ')[1])

['{']


In [78]:
import json
# >>> h = '{"foo":"bar", "foo2":"bar2"}'
d = json.loads(r.html.text.split('initialData = ')[1])
d

JSONDecodeError: Extra data: line 1 column 26854 (char 26853)

In [76]:
with open('text.json', 'w') as f:
    json.dump(r.html.text.split('initialData = ')[1], )

'{'

In [85]:
r.html.text.split('initialData = ')[1].split('images')[1]

'":{"meta":[{"src":"https://i.bikroy-st.com/oppo-a57-3-32-new-for-sale-barishal-7/5f986fc7-fd1b-4653-8129-36b15de96843","alt":"OPPO A57 3/32 (New) in Barishal"},{"src":"https://i.bikroy-st.com/oppo-a57-3-32-new-for-sale-barishal-7/01bc032e-2807-42dc-8f20-a2598bb8c703","alt":"OPPO A57 3/32 (New) in Nattullabad"},{"src":"https://i.bikroy-st.com/oppo-a57-3-32-new-for-sale-barishal-7/20c6bef2-26dd-4f86-bea5-7096b16a29cb","alt":"OPPO A57 3/32 (New) Mobile Phones"},{"src":"https://i.bikroy-st.com/oppo-a57-3-32-new-for-sale-barishal-7/6488d226-75f6-4025-a5b7-a28def255e96","alt":"OPPO A57 3/32 (New) "},{"src":"https://i.bikroy-st.com/oppo-a57-3-32-new-for-sale-barishal-7/67cc99ff-696f-4b5e-a951-0540798b12c8","alt":"OPPO A57 3/32 (New)"}],"base_uri":"https://i.bikroy-st.com","metadata":[]},"shop":{"id":"5e27f2d9f080fb000154c853","slug":"borisalshop","name":"Golden BD","email":"mdrajibfakir972@gmail.com","logo":"https://i.bikroy-st.com/u/borisalshop/9c404d5d-8874-4836-a544-ea8369e868b6/48/36/cro

In [91]:
r.html.text.split('image:')[0]

'OPPO A57 3/32 (New) in Nattullabad | Bikroy.com\nwindow.apiURL = "https://api.bikroy.com" window.chatConfig = {"enabled":true,"pubnubPublishKey":"pub-c-b1930cb0-7cdc-49f4-8b1e-a468cdee93a0","pubnubSubscribeKey":"sub-c-025a27be-cada-11e9-9b51-8ae91c2a8a9f","iOSBundleID":"se.saltside.bikroy"} window.initialData = {"locale":"en","marketLocales":["en","bn"],"locations":{"type":"NotAsked"},"categories":{"type":"NotAsked"},"category":{"id":230,"name":"Mobile Phones"},"browserInfo":{"fakeWidth":1200,"webpSupported":false},"jobsVertical":{"filters":{"type":"NotAsked"}},"serp":{"ads":{"type":"NotAsked"},"staticFilters":{"sort":"date_desc","posterType":"all_posters","isDoorstepDelivery":false,"isUrgent":false,"isFeaturedMember":false,"location":-1,"category":-1,"categorySlug":"","locationSlug":"bangladesh","page":1,"query":""},"dynamicFilters":{},"snapshot":null},"verticals":[{"key":"jobs","name":"Jobs","categories":[{"id":700},{"id":1401},{"id":1402},{"id":1404},{"id":1405},{"id":1406},{"id":1

In [39]:
def meta_data_scrap(r):
    try:
        meta_data = r.html.find('.ad-meta--17Bqm')[0].text

#         md = meta_data.find_elements(by=By.TAG_NAME,value='div')
#         meta_dic = {}
#         for m in md:
#             m = m.text
#             if '\n' in m:
#                 key, val = m.split('\n')[0].replace(':', ''), m.split('\n')[1]
#                 meta_dic[key] = val
#         print("Meta Data: ", meta_dic)
        return meta_dic
    except:
        print('No meta data found')        
        return 'N/A'

In [8]:
def show_more_click(driver):    
    try:
        s_driver.find_element(by=By.CLASS_NAME, value='show-text--13FcL').click()
#         s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[2]/button').click()
        time.sleep(2)
    except:
        print("No 'Show more' button")

In [41]:
def feature_scrap(r):
    try:
        feature = r.html.find('.text--Xap4O')[0].text
#         feature = s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[2]/div[4]/p').text
#         feature = feature.find_elements(by=By.TAG_NAME,value='p').text
        print('Features: ', feature)
        return feature
    except:
        print("No feature")        
        return 'N/A'

In [40]:
def description_scrap(r):
    # desc = s_driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[1]/div/ul/div')
    # description[0].text
    # description = [d.text.replace("'\uf076", "").replace("''\uf0d8", "") for d in desc.find_elements_by_tag_name('p')]
    # print(description)

    try:
        desc = r.html.find('.description-section--oR57b')[0].text
        print("Description: ", desc)
        return desc
    except:
        print("No description")        
        return 'N/A'

In [18]:
def product_scrap(r, category):
#     driver.get(url)
#     time.sleep(5)

    # single page data scrapping
    title = title_scrap(r)
    # if title not found (product sold) don't proceed further for other data
    if not title:
        return [None]*7
    # scrapping time and location
    time_location = time_location_scrap(r)
    
    # scrapping price
    price = price_scrap(r)
    
    # scrapping all images
    images = image_scrap(r, category)
    

    # extractig meta data
    meta_data = meta_data_scrap(r)
        
    # click on show_more button
    show_more_click(r)

    # extracting feature data
    feature = feature_scrap(r)
    
    # extracting description
    description = description_scrap(r)
    
    return title, time_location, price, images, meta_data, feature, description

In [34]:
category = 'mobile'
session = HTMLSession()

with open(f'category_url/{category}.txt', 'r') as f:
    single_url = f.readlines()
    titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst = [], [], [], [], [], [], []
    cnt = 1
#     print(single_url)
    for s in single_url[:2]:
        s = s.replace('\n', '')
        print(cnt, s)
        r = session.get('http://localhost:8050/render.html', params={'url': s, 'wait': 3})
        print(r.status_code)       
        title, time_location, price, images, meta_data, feature, description = product_scrap(r, category)
        if title:
            titles.append(title)
            locations.append(time_location)
            prices.append(price)
            img_lst.append(images)
            meta_lst.append(meta_data)
            feature_lst.append(feature)
            desc_lst.append(description) 

            cnt += 1
            print("=============================================")
            clear_output(wait=True)

1 https://bikroy.com/en/ad/oneplus-9-pro-12gb-256gb-global-new-for-sale-chattogram
200
200
OnePlus 9 Pro 12GB 256GB global (New)
Posted on 06 Mar 6:25 pm, CDA Avenue, Chattogram
Tk 63,990
Condition:
New
Brand:
OnePlus
Model:
9 Pro
Edition:
12GB 256GB global
Authenticity:
Original
Description
@@ new and pack❤️😍
14 days Replacement Guarantee and 1 year warranty.
🎯 Exchange possible_Best offer.
♻️Buy, Sell & Exchange♻️
💧Full Box with all Original Accessories.
📷 Original Picture Uploaded 📷
🔥We sell 100% Original & Non Refurbished Product.
📌 আপনার পুরনো স্মার্টফোনটি দিয়ে #EXCHANGE করে নতুন ফোন কিনতে পারবেন।
এছাড়াও আমরা পুরনো ফোন ক্রয়ও করে থাকি
14দিনের রিপ্লেস্মেন্ট ওয়ারেন্টি। ( MI তে ২৪ মাস সার্ভিস ওয়ারেন্টি)
আপনি আমাদের থেকে পাচ্ছেন ঃSAMSUNG,XIAOMI, Realme, NOKIA, vivo, oppo,, Apple , Motorola, মোবাইল
আমাদের সুবিধা সমূহ ঃ
১)অরিজিনাল প্রোডাক্ট এর গ্যারান্টি
২)১৪ দিন এর রিপ্লেসমেন্ট সুবিধা
৩)এক্সচেজ্ এর সুবিধা
৪) ৩- ৩৬মাস এর কিস্তি সুবিধা
আপনাদের জন্য মারবেল দিচ্ছে ৩,৬,৯,১২,১৮,২৪,৩৬,মাস এর ক

AttributeError: 'HTMLResponse' object has no attribute 'find_element'

In [47]:
r = session.get('http://localhost:8050/render.html', params={'url': s, 'wait': 3})
r

<Response [200]>

In [49]:
print(s)

https://bikroy.com/en/ad/oneplus-9-pro-12gb-256gb-global-new-for-sale-chattogram


In [19]:
df_lst = [titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst, single_url[:2]]
column_lst = ['Title', 'Location', 'Price', 'Images', 'Meta Data', 'Feature', 'Description', 'URL']
df = pd.DataFrame (df_lst).T
df.columns = column_lst
df.to_csv(f'csv/{category}.csv', index=False)
df.head()


,Title,Location,Price,Images,Meta Data,Feature,Description,URL
0,Realme C11 (Used),"Posted on 07 Mar 11:15 pm, Maulvi Bazar, Sylhe...","Tk 6,000",[realme-c11-used-for-sale-sylhet-division-12@1...,"{'Condition': 'Used', 'Brand': 'Realme', 'Mode...",N/A,N/A,https://bikroy.com/en/ad/oneplus-9-pro-12gb-25...
1,Samsung Galaxy A7 . (Used),"Posted on 07 Mar 11:15 pm, Dhanmondi, Dhaka","Tk 9,000",[samsung-galaxy-a7-2018-used-for-sale-dhaka-11...,"{'Condition': 'Used', 'Brand': 'Samsung', 'Mod...","Bluetooth, Camera, Dual SIM, Expandable Memory...",N/A,https://bikroy.com/en/ad/oppo-a57-3-32-new-for...


In [20]:
titles

['Realme C11 (Used)', 'Samsung Galaxy A7 . (Used)']